In [1]:
# -----------------------------
# MNIST DIGIT CLASSIFIER (PyTorch)
# -----------------------------

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from PIL import ImageOps
import gradio as gr

In [2]:
# -----------------------------
# 1. LOAD DATA
# Transforms are preprocessing steps that get applied automatically to every image
# you load from a dataset.
# Think of transforms as a recipe that says:

# “Every time you give me an image, do X, then Y, then Z to it.”
# “For every MNIST image: convert it to a PyTorch tensor.
# MNIST images come in as PIL images (Python Imaging Library).

# But your neural network expects tensors.
# -----------------------------
transform = transforms.Compose([
    transforms.ToTensor()
])

In [3]:
# Load training dataset (MNIST)
train_dataset = datasets.MNIST(
    root="./data",
    train=True,
    transform=transform,
    download=True
)


100%|██████████| 9.91M/9.91M [00:00<00:00, 43.8MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.09MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 10.1MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.71MB/s]


In [4]:
# Load test dataset
test_dataset = datasets.MNIST(
    root="./data",
    train=False,
    transform=transform,
    download=True
)


In [6]:
# Make DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

# TODO: Access and print the unique labels in the training data set using the train_loader object

unique_labels = set()

for _, labels in train_loader:
    unique_labels.update(labels.tolist())

print("Unique labels in training set:", sorted(unique_labels))



Unique labels in training set: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [7]:
# -----------------------------
# 2. DEFINE NEURAL NETWORK
# TODO: Design a Neural Network with 1 hidden layer of 128 neurons
# -----------------------------
class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()

        # TODO: Define layers
        self.fc1 = nn.Linear(28 * 28, 128)  # input → hidden
        self.fc2 = nn.Linear(128, 10)       # hidden → output (10 classes)

    def forward(self, x):
        # Flatten image: (batch, 1, 28, 28) → (batch, 784)
        x = x.view(-1, 28 * 28)

        # TODO: Add activation between layers
        x = torch.relu(self.fc1(x))

        # TODO: Output layer
        x = self.fc2(x)

        return x


In [8]:
# TODO: Create the model

model = SimpleNN()
print(model)


SimpleNN(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


In [9]:
# -----------------------------
# 3. LOSS FUNCTION + OPTIMIZER
# -----------------------------
# TODO: Define your loss function

criterion = nn.CrossEntropyLoss()

# TODO: Setup your gradient descent . Try different values for the learning rate

optimizer = optim.SGD(model.parameters(), lr=0.01)


In [10]:
# -----------------------------
# 4. TRAINING LOOP
# -----------------------------

# TODO: Define the number of epochs
epochs = 5

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        # TODO: Reset the gradients
        optimizer.zero_grad()

        # TODO: Forward pass
        outputs = model(images)

        # TODO: Compute loss
        loss = criterion(outputs, labels)

        # TODO: Backpropagate
        loss.backward()

        # TODO: Update gradients
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


Epoch 1, Loss: 1178.5393
Epoch 2, Loss: 464.3172
Epoch 3, Loss: 367.8100
Epoch 4, Loss: 329.8498
Epoch 5, Loss: 306.6048


In [11]:
# -----------------------------
# 5. EVALUATION
# -----------------------------
correct = 0
total = 0
model.eval()

with torch.no_grad():
    for images, labels in test_loader:
        # TODO: Forward pass
        outputs = model(images)

        # Predicted class = index of max logit
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 91.61%


In [16]:
# -----------------------------
# 6. TEST SINGLE PREDICTION + UI
# -----------------------------
# Gradio Sketchpad gives:
# - a color NumPy array
# - big resolution
# - black digit on white background
# We need to convert it to a 28x28 MNIST-like tensor.

def preprocess_image(image):
    sketch_transform = transforms.Compose([
        transforms.ToPILImage(),                             # NumPy → PIL
        transforms.Grayscale(),                              # 3-channel → 1-channel
        transforms.Resize((28, 28)),                         # 28x28 like MNIST
        transforms.Lambda(lambda img: ImageOps.invert(img)), # make background black, digit white
        transforms.ToTensor(),                               # → tensor, shape (1, 28, 28)
    ])

    # Sometimes Sketchpad returns a dict: {"composite": np.array}
    if isinstance(image, dict):
        image = image["composite"]

    # Apply transform
    img_tensor = sketch_transform(image)      # (1, 28, 28)

    # Add batch dimension → (1, 1, 28, 28)
    img_tensor = img_tensor.unsqueeze(0)

    return img_tensor


def predict_digit(image):
    # 1. Nothing drawn
    if image is None:
        return "Draw something!"

    # 2. Preprocess
    img_tensor = preprocess_image(image)

    # 3. Run model
    model.eval()
    with torch.no_grad():
        prediction = model(img_tensor)
        predicted_digit = torch.argmax(prediction).item()

    return str(predicted_digit)


# UI
interface = gr.Interface(
    fn=predict_digit,
    inputs=gr.Sketchpad(label="Draw Here"),
    outputs="label"
)

interface.queue().launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bfbb6179dcf60f5387.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
